In [17]:
case_url = r"E:\Lawnics\Court Cases\Supreme_Court\2019\August\Ajith K. vs Aneesh K.S. on 21 August, 2019.html"

In [1]:
import jsonlines
import string 
import pandas as pd
import json
# import glob
import random
import re
from bs4 import BeautifulSoup
import os
#Spacy imports
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy
from collections import Counter
from spacy.util import minibatch, compounding
from spacy.util import decaying

import warnings
import pickle
import plac
from pathlib import Path
from tqdm import tqdm

spacy.prefer_gpu()

False

In [2]:
cite_dict = {}
path_to_parent = r"E:/Lawnics/Court Cases/Supreme_Court"
for year in tqdm(os.listdir(path_to_parent)):
    if len(year) <= 4:
        for month in os.listdir(path_to_parent+"/"+year):
            for case in os.listdir(path_to_parent+"/"+year+"/"+month):
                if case.endswith("json"):
                    x = str(os.path.join(path_to_parent, year,month,case))
                    with open(x, 'r') as j:
                        contents = json.loads(j.read())
                        for item in contents["cites"]:
                            if "v" in item.lower() or "vs" in item and "act" not in item and "section" not in item and "article" not in item:
                                cite_dict[item] = "0"

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [11:52<00:00,  9.13s/it]


In [102]:
cite_dict
for k in cite_dict.copy():
    x = k.lower()
    if "section" in x or "act" in x or "article" in x:
        cite_dict.pop(k)
cite_dict
new_dict = {}
for k in cite_dict:
    res = " ".join(remove_punc(k).split())
    string = res.split(" on ")[0]
    new_dict[string] = "0"
    
new_dict

{'the secretary ministry of vs cricket association of bengal': '0',
 'pakala narayana swami vs emperor': '0',
 'hardeep singh vs state of punjab ors': '0',
 'laxman moreshwar mahurkar vs balkrishna jagannath kinikar and': '0',
 'eradu and ors vs state of hyderabad': '0',
 'lakhanpal vs state of madhya pradesh': '0',
 'secretary of state vs firm daulat rammakhan lal': '0',
 'cbiacbmumbai vs narendra lal jain ors': '0',
 'ic golak nath and ors vs state of punjab and anr': '0',
 'kailash chand sharma etc etc vs state of rajasthan and ors': '0',
 'mohd shafi vs additional district and sessions': '0',
 'm ahamedkutty vs union of india anr': '0',
 'moti singh vs state of maharashtra': '0',
 'dominic varkey vs the state of kerala': '0',
 'madan lal ors vs the state of jammu kashmir and ors': '0',
 'b ajodhya prasad singh kumar and vs b kamal narain singh and ors': '0',
 'k kamaraja nadar vs kunju thevar and ors': '0',
 'sheo swarup vs kingemperor': '0',
 'khajeh solehman quadir vs nawab sir s

In [88]:
len(new_dict)

39091

In [4]:
# def remove_punc(text):
#     punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
#     no_punct = ""
#     for char in text:
#         if char not in punctuations:
#             no_punct = no_punct + char

#     # display the unpunctuated string
#     return no_punct

In [2]:
def create_text(text):
    string = ""
    rr_soup=BeautifulSoup(text)
    res = rr_soup.find_all('p')
    for para in res:
        ntext = para.text.replace("\n", " ")
        
        string = string + ntext
    res = " ".join(string.split())
    return res

In [135]:
def create_dict(text):
    
    dictionary = {}
    lst = []
    b_lst = []
    rr_soup=BeautifulSoup(text)
    for item in rr_soup.find_all('blockquote'):
        b_lst.append(item.find_all("a"))
        for link in b_lst:
            for text in link:
                dictionary[text.text] = "0"
    for item in rr_soup.find_all('p'):
        lst.append(item.find_all("a"))
        for link in lst:
            for text in link:
                dictionary[text.text] = "0"
    dictionary.update(new_dict)
#     print(dictionary)
    return dictionary

In [48]:
def create_entities(dictionary, text):
    text = remove_punc(text).lower()
    new_lst = []
    new_dict = {}
#     print(dictionary,text)
    for key in dictionary:
        key = remove_punc(key).lower()
        if "Act" in key or "act" in key:
            start = text.find(key)
            if start != -1:
                new_lst.append(((start,start+len(key),"Act")))
        if "Section" in key or "section" in key:
            start = text.find(key)
            if start != -1:
                new_lst.append(((start,start+len(key),"Section")))
        elif "Article" in key or "article" in key:
            start = text.find(key)
            if start != -1:
                new_lst.append((start,start+len(key),"Article"))
        elif "v." in key or "V." in key or "vs." or "vs" in key:
            if len(key) > 20:
                start = text.find(key)
                if start != -1:
                    new_lst.append((start,start+len(key),"Cases"))
        else:
            start = text.find(key)
            if start != -1:
                new_lst.append((start,start+len(key),"Other"))
    new_dict["entities"] = new_lst
#     print(new_dict)
    return new_dict

In [37]:
def create_dataset(url, count):
    f = open(url, encoding = "utf8")
    case = f.read()
    sptext = create_text(case)
    dictionary = create_dict(case)
    entities = create_entities(dictionary, sptext)
    sptext = remove_punc(sptext).lower()
    spacy_tuple = (sptext,entities)
#     print(sptext,entities)
    return spacy_tuple

In [138]:
base = r"E:/Lawnics/Court Cases"
num = 1
lst = []
for court in os.listdir(base):
    if court == "Supreme_Court":
        for year in tqdm(os.listdir(base+"/"+court)):
            if year.endswith(".json"):
                continue
            check = int(year)
            if check >= 2012:
                for month in os.listdir(base+"/"+court+"/"+year):
                    for case in os.listdir(base+"/"+court+"/"+year+"/"+month):
                        if case.endswith("html"):
                            res_dict = create_dataset(base+"/"+court+"/"+year+"/"+month+"/"+case,num)
                            num += 1
                            lst.append(res_dict)
#                     print(case)

100%|████████████████████████████████████████████████████████████████████████████████| 78/78 [1:09:57<00:00, 53.82s/it]


In [79]:
len(lst)

2643

In [50]:
create_dataset(case_url,1)


('try out our premium member services virtual legal assistant query alert service and an adfree experience free for one month and pay only if you like it dr dhananjaya y chandrachud j 1 this batch of appeals arises from a judgment dated 10 december 2018 of a division bench of the high court of kerala affirming the correctness of the judgment of the kerala administrative tribunal dated 20 december 2017 the high signature not verified court dismissed the writ petition filed by the appellants under article 227 of the digitally signed by sanjay kumar date 20190821 163255 ist reason constitution 1 2 on 16 august 1972 the government of kerala in exercise of its powers under section 11 of the kerala municipal common service rules 1967 specified the method of recruitment and qualifications for posts in the municipal common service the posts were specified in an annexure to the order the post of health inspector  food inspector gradeii was among those posts and the qualifications were 3 on 26 d

In [139]:
count = 1
with jsonlines.open('output1.jsonl', mode='w') as writer:
    for item in tqdm(lst):
        t_dict= {}
        t_dict["id"] = count
        t_dict["text"] = item[0]
        t_dict["labels"] = item[1]["entities"]
        writer.write(t_dict)
        count += 1

100%|████████████████████████████████████████████████████████████████████████████| 7774/7774 [00:01<00:00, 4751.84it/s]


In [ ]:
item

In [ ]:
train_set = lst[:1500]
len(train_set)

In [ ]:
test_set = lst[1500:]
len(test_set)

In [99]:
string = "the secretary ministry of vs cricket association of bengal on 9 february 1995"
string.split(" on ")

['the secretary ministry of vs cricket association of bengal',
 '9 february 1995']

In [124]:
x = create_text(case_url)
asd = create_dict(x)

In [127]:
f = open(case_url, encoding = "utf8")
case = f.read()

In [136]:
f = open(case_url, encoding = "utf8")
case = f.read()
sptext = create_text(case)
dictionary = create_dict(case)
entities = create_entities(dictionary, sptext)
sptext = remove_punc(sptext).lower()
spacy_tuple = (sptext,entities)

In [137]:
spacy_tuple

('try out our premium member services virtual legal assistant query alert service and an adfree experience free for one month and pay only if you like it dr dhananjaya y chandrachud j 1 this batch of appeals arises from a judgment dated 10 december 2018 of a division bench of the high court of kerala affirming the correctness of the judgment of the kerala administrative tribunal dated 20 december 2017 the high signature not verified court dismissed the writ petition filed by the appellants under article 227 of the digitally signed by sanjay kumar date 20190821 163255 ist reason constitution 1 2 on 16 august 1972 the government of kerala in exercise of its powers under section 11 of the kerala municipal common service rules 1967 specified the method of recruitment and qualifications for posts in the municipal common service the posts were specified in an annexure to the order the post of health inspector  food inspector gradeii was among those posts and the qualifications were 3 on 26 d